In [1]:
import re
from collections import deque

In [2]:
# adapted from: https://csl.name/post/vm/


class Stack(deque):

    push = deque.append

    def peek(self):
        return self[-1]

In [3]:
class Interpreter:
    def __init__(self, code: list):

        self.values_stack = Stack()
        self.operations_stack = Stack()
        self.instruction_pointer = 0
        self.code = code
        self.op_map = {
            "*": self.mul,
            "+": self.plus,
        }

    def run(self):
        while self.instruction_pointer < len(self.code):
            token = self.code[self.instruction_pointer]
            self.instruction_pointer += 1
            self.do(token)

    def do(self, token):

        # print(f"Doing token {token}")

        if token in self.op_map:
            self.operations_stack.push(token)

        elif isinstance(token, int):
            self.values_stack.push(token)

        else:
            raise RuntimeError(f"Unknown opcode: '{token}'")

        self.should_calculate()

    def should_calculate(self):

        # print("Values:", self.values_stack)
        # print("Ops:", self.operations_stack)

        if len(self.values_stack) == 2:

            # print("Calculating...")
            assert len(self.operations_stack) == 1

            op = self.operations_stack.pop()

            self.op_map[op]()

            # print("New values:", self.values_stack)

    def plus(self):
        v1 = self.values_stack.pop()
        v2 = self.values_stack.pop()
        self.values_stack.push(v1 + v2)

    def mul(self):
        v1 = self.values_stack.pop()
        v2 = self.values_stack.pop()
        # print(v1,v2)
        self.values_stack.push(v1 * v2)

    def result(self):
        assert len(self.values_stack) == 1
        return self.values_stack.pop()

In [153]:
def parse_code(code: str):

    str_tokens = code.split(" ")  # list(code.replace(" ", ""))

    tokens = []

    for t in str_tokens:
        if t in {"*", "+"}:
            tokens.append(t)
        else:
            tokens.append(int(t))

    return tokens

In [155]:
code = parse_code("1 + 2 + 3 * 5")
code

[1, '+', 2, '+', 3, '*', 5]

In [58]:
i = Interpreter(code)

In [59]:
i.run()

In [60]:
i.result()

30

In [61]:
def interpret_code(code: str):
    
    code = parse_code(code)
    
    i = Interpreter(code)
    
    try:
        i.run()
    except:
        print(i.code)
    
    return i.result()

In [62]:
orig = "1 + 2 + (1 + (4 + 4)) + (1 + (4 + 4))"
m = re.search(r"\(([^\(|\)]*)\)", orig)
m

<re.Match object; span=(13, 20), match='(4 + 4)'>

In [63]:
m.group(1)

'4 + 4'

In [64]:
print(m.span())
expr = m.group(1)
expr

(13, 20)


'4 + 4'

In [65]:
interpret_code(expr)

8

In [66]:
from typing import Tuple

In [67]:
def find_paren_group(code: str) -> Tuple[str, Tuple[int, int]]:

    m = re.search(r"\(([^\(|\)]*)\)", code)

    expr = m.group(1)

    return expr, m.span()

In [68]:
def replace_str(orig, span, new):
    
    return str(orig[:span[0]]) + str(new) + str(orig[span[1]:])

In [84]:
def remove_paren(code: str):
    
    while "(" in code or ")" in code:
        
        expr, span = find_paren_group(code)
        # print(expr, span)
        
        value = interpret_code(expr)
        
        code = replace_str(code, span, value)
        
    return code

In [85]:
def do(code: str):
    code = remove_paren(code)
    
    return interpret_code(code)

**Examples given**

In [88]:
assert do("5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))") == 12240
assert do("2 * 3 + (4 * 5)") == 26
assert do("5 + (8 * 3 + 9 + 3 * 4 * 3)") == 437
assert do("((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2") == 13632

In [92]:
with open("input.txt") as f:
    data = f.read().split("\n")

In [95]:
sum([do(x) for x in data])

11076907812171

## Part 2

In [146]:
class Interpreter:
    def __init__(self, code: list):

        self.values_stack = Stack()
        self.operations_stack = Stack()
        self.instruction_pointer = 0
        self.code = code

        self.op_map = {"*": self.mul, "+": self.plus}
        self.op_precendence = {"+": 1, "*": 0}

    def run(self):
        while self.instruction_pointer < len(self.code):
            token = self.code[self.instruction_pointer]
            self.instruction_pointer += 1
            self.do(token)

    def do(self, token):

        # print(f"Doing token {token}")

        if token in self.op_map:
            if len(self.operations_stack) == 0:
                self.operations_stack.push(token)
            else:
                # print("doing while")
                
                while (len(self.operations_stack) > 0) and (
                    self.op_precendence[self.operations_stack.peek()]
                    > self.op_precendence[token]
                ):
                    
                    # print(f"Ops: {self.operations_stack}")
                    self.process()

                    
                # print(f"finished process, pushing token {token}")

                self.operations_stack.push(token)

        elif isinstance(token, int):
            self.values_stack.push(token)

        else:
            raise RuntimeError(f"Unknown opcode: '{token}'")

        # self.should_calculate()

    def process(self):

        # print("Values:", self.values_stack)
        # print("Ops:", self.operations_stack)

        op = self.operations_stack.pop()

        self.op_map[op]()

        # print("New values:", self.values_stack)

    def plus(self):
        v1 = self.values_stack.pop()
        v2 = self.values_stack.pop()
        # print(v1, v2)
        self.values_stack.push(v1 + v2)

    def mul(self):
        v1 = self.values_stack.pop()
        v2 = self.values_stack.pop()
        # print(v1, v2)
        self.values_stack.push(v1 * v2)

    def result(self):
        while len(self.values_stack) > 1:
            self.process()

        return self.values_stack.pop()

In [147]:
def interpret_code(code: str):

    code = parse_code(code)

    i = Interpreter(code)

    try:
        i.run()
    except:
        print(i.code)
        raise

    return i.result()

In [150]:
interpret_code("1 + 2 * 4 * 5 + 1 * 2")

144

**Examples given**

In [151]:
examples = [
    ("1 + (2 * 3) + (4 * (5 + 6))", 51),
    ("2 * 3 + (4 * 5)", 46),
    ("5 + (8 * 3 + 9 + 3 * 4 * 3)", 1445),
    ("5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))", 669060),
    ("((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2", 23340),
]


for code, result in examples:
    assert do(code) == result

In [152]:
sum([do(x) for x in data])

283729053022731